In [5]:
# importar las librerias que se van a usra
using JuMP
using Ipopt 
using HiGHS 
using Optimization

In [ ]:
model = Model()

# Variables de producción e inventario
@variable(model, 0 <= x[1:6] <= 225)  # Producción por mes con  limite de 255 
@variable(model, I[0:6] >= 0)  # Inventario inicial (I0) y final de cada mes
fix(I[0], 0; force=true)  # Inventario inicial = 0

# Demandas por mes
D = [180, 250, 190, 140, 220, 250]

# Restricciones de balance de inventario
for i in 1:6
    # elel vaalor de I_i se compone de lo producido en el mes i + 
    #lo que quedó del mes anterior - la demanda del mes correspondiente 
    @constraint(model, I[i] == I[i-1] + x[i] - D[i])
end

costos_prod = [50, 45, 55, 52, 48, 50]
costos_inv = [8, 10, 10, 10, 8, 8]  
# Función objetivo (costos producción + almacenamiento)
@objective(model, Min, sum(costos_prod[i] * x[i] for i in 1:6) + 
                       sum(costos_inv[i] * I[i] for i in 1:6))



50 x[1] + 45 x[2] + 55 x[3] + 52 x[4] + 48 x[5] + 50 x[6] + 8 I[1] + 10 I[2] + 10 I[3] + 10 I[4] + 8 I[5] + 8 I[6]

In [7]:
set_optimizer(model, HiGHS.Optimizer)

In [8]:
model

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 13
├ num_constraints: 25
│ ├ AffExpr in MOI.EqualTo{Float64}: 6
│ ├ VariableRef in MOI.EqualTo{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 12
│ └ VariableRef in MOI.LessThan{Float64}: 6
└ Names registered in the model
  └ :I, :x

In [9]:
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 13 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [2e+02, 2e+02]
  RHS    [1e+02, 2e+02]
Presolving model
6 rows, 12 cols, 17 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
2 rows, 7 cols, 8 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
2 rows, 7 cols, 8 nonzeros  0s
Presolve : Reductions: rows 2(-4); columns 7(-6); elements 8(-10)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.6295000000e+04 Pr: 2(800) 0s
          3     6.1795000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  6.1795000000e+04
P-D objective

In [10]:
# Resultados
println("Producción óptima: ", value.(x))
println("Inventario óptimo: ", value.(I[1:6]))
println("Costo total mínimo: \$", objective_value(model))

Producción óptima: [205.0, 225.0, 190.0, 160.0, 225.0, 225.0]
Inventario óptimo: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6]
And data, a 6-element Vector{Float64}:
 25.0
  0.0
  0.0
 20.0
 25.0
  0.0
Costo total mínimo: $61795.0


## MOdelo sin inventario

In [ ]:
model_no_inventory = Model(HiGHS.Optimizer)

@variable(model_no_inventory, 0 <= x[1:6] <= 250)  # Capacidad de 250
D = [180, 250, 190, 140, 220, 250]  # Demandas

# Restricción: producir exactamente la demanda
for i in 1:6
    @constraint(model_no_inventory, x[i] == D[i])
end

@objective(model_no_inventory, Min, sum([50, 45, 55, 52, 48, 50][i] * x[i] for i in 1:6))

optimize!(model_no_inventory)

println("Producción (sin inventario): ", value.(x))
println("Costo total: \$", objective_value(model_no_inventory))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 6 cols; 6 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e+01, 6e+01]
  Bound  [2e+02, 2e+02]
  RHS    [1e+02, 2e+02]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-6); columns 0(-6); elements 0(-6) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  6.1040000000e+04
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01
Producción (sin inventario): [180.0, 250.0, 190.0, 140.0, 220.0, 250.0]
Costo total: $61040.0


## Ahora con restricciones enteras

In [ ]:
model_enteros = Model()

# Variables de producción e inventario pero con restriccion entera
@variable(model_enteros, 0 <= x[1:6] <= 225, Int)  
@variable(model_enteros, I[0:6] >= 0, Int)  
fix(I[0], 0; force=true)  

# Demandas por mes
D = [180, 250, 190, 140, 220, 250]

# Restricciones de balance de inventario
for i in 1:6
    @constraint(model_enteros, I[i] == I[i-1] + x[i] - D[i])
end

costos_prod = [50, 45, 55, 52, 48, 50]
costos_inv = [8, 10, 10, 10, 8, 8]  
@objective(model_enteros, Min, sum(costos_prod[i] * x[i] for i in 1:6) + 
                       sum(costos_inv[i] * I[i] for i in 1:6))



50 x[1] + 45 x[2] + 55 x[3] + 52 x[4] + 48 x[5] + 50 x[6] + 8 I[1] + 10 I[2] + 10 I[3] + 10 I[4] + 8 I[5] + 8 I[6]

In [13]:
set_optimizer(model_enteros, HiGHS.Optimizer)

In [14]:
model_enteros

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 13
├ num_constraints: 38
│ ├ AffExpr in MOI.EqualTo{Float64}: 6
│ ├ VariableRef in MOI.EqualTo{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 12
│ ├ VariableRef in MOI.LessThan{Float64}: 6
│ └ VariableRef in MOI.Integer: 13
└ Names registered in the model
  └ :I, :x

In [15]:
optimize!(model_enteros)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 6 rows; 13 cols; 18 nonzeros; 13 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [2e+02, 2e+02]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 9 cols, 11 nonzeros  0s
2 rows, 6 cols, 7 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQu

In [16]:
# Resultados
println("Producción óptima: ", value.(x))
println("Inventario óptimo: ", value.(I[1:6]))
println("Costo total mínimo: \$", objective_value(model_enteros))

Producción óptima: [205.0, 225.0, 190.0, 160.0, 225.0, 225.0]
Inventario óptimo: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6]
And data, a 6-element Vector{Float64}:
 25.0
  0.0
  0.0
 20.0
 25.0
  0.0
Costo total mínimo: $61795.0
